In [1]:
#install pyreadr on terminal
! pip install pyreadr lightgbm

     |████████████████████████████████| 437 kB 26.0 MB/s            
     |████████████████████████████████| 1.7 MB 58.3 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lightgbm: filename=lightgbm-4.0.0-py3-none-manylinux_2_27_x86_64.whl size=2462978 sha256=b3376d5f0420fd953d805750577446f99f505c8c3b411b5e0fe5a446b8fa2a8c
  Stored in directory: /home/ubuntu/.cache/pip/wheels/e3/b8/97/5e260d77a9cfac3783b63bc55226ba7e02db270a2779635d26
Successfully built lightgbm
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os
from termcolor import colored

from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.metrics import confusion_matrix,precision_recall_curve, roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score, roc_auc_score

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier 
from sklearn.ensemble import BaggingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC,SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN

# Haplotype model - Categorical variable

In [3]:
import pyreadr

# read in female data
result = pyreadr.read_r('/mnt/ML_HBLUP/NA_RM105_110_115/data/dummyMatrix_female.rds') # also works for RData
# done! 
# result is a dictionary where keys are the name of objects and the values python
# objects. In the case of Rds there is only one object with None as key
femaleData = result[None] # extract the pandas data frame 

# read in male data
result = pyreadr.read_r('/mnt/ML_HBLUP/NA_RM105_110_115/data/dummyMatrix_male.rds') # also works for RData
maleData = result[None] # extract the pandas data frame 

In [4]:
femaleData.head()

,HB1__1067-1,HB1__32843,HB1__64DWA2,HB1__B73,HB1__MANS,HB1__NA,HB1__WDAQ2,HB2__1067-1,HB2__32843,HB2__64DWA2,...,HB17115__7797,HB17115__B73,HB17115__NA,HB17115__RQAA8,HB17116__2FACC,HB17116__7797,HB17116__B73,HB17116__FBMU,HB17116__NA,HB17116__RQAA8
01DHD10,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DHD16,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DKD2-BGL-T1A1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2-NQR-T1B1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [5]:
maleData.head()

,HB1__01HGI4,HB1__610,HB1__B14,HB1__LH287,HB1__M3AG-3,HB1__NA,HB1__OH43AE1,HB1__PH207,HB2__610,HB2__B14,...,HB17114__OH07,HB17114__PH207,HB17115__LH123,HB17115__NA,HB17115__OH07,HB17115__TA1180,HB17116__LH123,HB17116__NA,HB17116__OH07,HB17116__PH207
LH287,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
83INI14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
17IFI6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
DILU757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GEJO564,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [6]:
# add suffix
femaleData.columns += '_f'
maleData.columns += '_m'

In [7]:
maleData.head()

,HB1__01HGI4_m,HB1__610_m,HB1__B14_m,HB1__LH287_m,HB1__M3AG-3_m,HB1__NA_m,HB1__OH43AE1_m,HB1__PH207_m,HB2__610_m,HB2__B14_m,...,HB17114__OH07_m,HB17114__PH207_m,HB17115__LH123_m,HB17115__NA_m,HB17115__OH07_m,HB17115__TA1180_m,HB17116__LH123_m,HB17116__NA_m,HB17116__OH07_m,HB17116__PH207_m
LH287,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
83INI14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
17IFI6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
DILU757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GEJO564,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [8]:
femaleData.head()

,HB1__1067-1_f,HB1__32843_f,HB1__64DWA2_f,HB1__B73_f,HB1__MANS_f,HB1__NA_f,HB1__WDAQ2_f,HB2__1067-1_f,HB2__32843_f,HB2__64DWA2_f,...,HB17115__7797_f,HB17115__B73_f,HB17115__NA_f,HB17115__RQAA8_f,HB17116__2FACC_f,HB17116__7797_f,HB17116__B73_f,HB17116__FBMU_f,HB17116__NA_f,HB17116__RQAA8_f
01DHD10,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DHD16,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DKD2-BGL-T1A1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2-NQR-T1B1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [9]:
allHapFemales = femaleData.index
allHapMales = maleData.index

In [35]:
# read in the train data and test data
trainPheno = pd.read_csv('/mnt/ML_HBLUP/NA_RM105_110_115/data/train_phenoData_NA_Corn_hblup_2015-2020_ALL_UDR_105-110-115.csv')
testPheno = pd.read_csv('/mnt/code/pred_2020/NA_Corn_hblup_2020_ALL_UDR_105-110-115_RM.csv')

In [36]:
trainPheno.head()

,LINE_NAME,FEMALE,MALE,YLD_BE_BLUP,MST_BE_BLUP
0,01DHD10+LH287,01DHD10,LH287,-15.661,-0.219
1,01DHD16+83INI14,01DHD16,83INI14,-41.839,-4.254
2,01DKD2-BGL-T1A1+17IFI6,01DKD2-BGL-T1A1,17IFI6,-30.021,-3.251
3,01DKD2-BGL-T1A1+DILU757,01DKD2-BGL-T1A1,DILU757,8.279,1.769
4,01DKD2-BGL-T1A1+GEJO564,01DKD2-BGL-T1A1,GEJO564,4.171,1.519


In [37]:
testPheno.head()

,LINE_NAME,FEMALE,MALE,YLD_BE_BLUP
0,JEDE424+JYLV1963,JEDE424,JYLV1963,-0.983249
1,GIDA1530+HILW1982,GIDA1530,HILW1982,-0.008326
2,HUKI1698+DINO359,HUKI1698,DINO359,1.929877
3,JYDD1826+GALV1529,JYDD1826,GALV1529,-1.938917
4,HIQO1921+HULU1619,HIQO1921,HULU1619,-2.036041


In [38]:
# filter trainData by selecting hybrids with both female and male are included in haplotypdata
# and Observation is not NA
trainPheno = trainPheno[(trainPheno['FEMALE'].isin(allHapFemales)) & trainPheno['MALE'].isin(allHapMales) & trainPheno['YLD_BE_BLUP'].notna()]
testPheno = testPheno[testPheno['FEMALE'].isin(allHapFemales) & testPheno['MALE'].isin(allHapMales) & testPheno['YLD_BE_BLUP'].notna()]

# drop duplicated rows
trainPheno = trainPheno.drop_duplicates()
testPheno = testPheno.drop_duplicates()

In [39]:
print(trainPheno.shape)
print(testPheno.shape)

(33475, 5)
(9242, 4)


In [40]:
# filter trainPheno to remove overlapped lines with testPheno
testPheno = trainPheno[trainPheno['LINE_NAME'].isin(testPheno['LINE_NAME'])]
trainPheno = trainPheno[~trainPheno['LINE_NAME'].isin(testPheno['LINE_NAME'])]

In [41]:
print(trainPheno.shape)
print(testPheno.shape)

(24236, 5)
(9239, 5)


In [22]:
# construct haplotype data for test and train data
trainHap = pd.concat([femaleData.loc[trainPheno['FEMALE'],:].reset_index(drop=True),
                      maleData.loc[trainPheno['MALE'],:].reset_index(drop=True)],axis=1)
trainHap = trainHap / 2

testHap = pd.concat([femaleData.loc[testPheno['FEMALE'],:].reset_index(drop=True),
                     maleData.loc[testPheno['MALE'],:].reset_index(drop=True)],axis=1)
testHap = testHap / 2

In [23]:
# splite the train data into train, validationa and test
seed = 20230510
np.random.seed(seed)
X_train, X_val, y_train, y_val = train_test_split(trainHap, trainPheno['YLD_BE_BLUP'], test_size=0.1, random_state=seed)


In [45]:
print(X_train.shape)
print(X_val.shape)
print(testHap.shape)

(21812, 109598)
(2424, 109598)
(9239, 109598)


In [51]:
# a function to return correlation and rmse
from scipy.stats import pearsonr

metrics = pd.DataFrame(columns=['Method', 'RMSE_train', 'RMSE_val','RMSE_test','corr_train','corr_val','corr_test'])
def pred_rmse_corr(modelRes,modelName,X_train,y_train,X_val,y_val,X_test,y_test,metrics):
    pred_train = modelRes.predict(X_train)
    pred_val = modelRes.predict(X_val)
    pred_test = modelRes.predict(X_test)

    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    rmse_val = mean_squared_error(y_val,pred_val, squared=False)
    rmse_test = mean_squared_error(y_test, pred_test, squared=False)

    corr_train, p_value_train = pearsonr(y_train.ravel(), pred_train.ravel())
    corr_val, p_value_val = pearsonr(y_val.ravel(), pred_val.ravel())
    corr_test, p_value_test = pearsonr(y_test.ravel(), pred_test.ravel())


    metrics_curr_cv = pd.DataFrame(data={'Method': modelName, 'RMSE_train': [rmse_train], 'corr_train': [corr_train],
                                     'RMSE_val' : [rmse_val], 'corr_val' : [corr_val], 
                                     'RMSE_test' : [rmse_test], 'corr_test' : [corr_test]})

    metrics = pd.concat([metrics, metrics_curr_cv], axis=0)
    return metrics

## LightGBM

In [26]:
# need to install lightgbm from terminal: sudo pip install lightgbm
import lightgbm as lgb

gbm = lgb.LGBMRegressor(num_leaves=30,
                        learning_rate=0.1,
                        n_estimators=900,
                       bagging_fraction =  0.7,
                       feature_fraction = 0.5,
                       objective = "regression")

In [28]:
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='mse',
        callbacks=[lgb.early_stopping(5)])



# load model later
    #model = lightgbm.Booster(model_file='file.txt')
    #model.predict(predict[num_columns])
    

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 18.374392 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 275400
[LightGBM] [Info] Number of data points in the train set: 21812, number of used features: 109579
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value:

LGBMRegressor(bagging_fraction=0.7, feature_fraction=0.5, n_estimators=900,
              num_leaves=30, objective='regression')

In [29]:
# save model
gbm.booster_.save_model('/mnt/code/pred_2020/models/hap_lgbm_2020.txt')

# load the saved model
# gbm = lgb.Booster(model_file='/mnt/ML_HBLUP/NA_RM105_110_115/models/hap_lgbm_2020.txt')

In [52]:
# output prediction result
metrics = pred_rmse_corr(gbm,'LightGBM_hapCat',X_train,y_train,
                         X_val,y_val,testHap,testPheno['YLD_BE_BLUP'],metrics)
metrics

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7


,Method,RMSE_train,RMSE_val,RMSE_test,corr_train,corr_val,corr_test
0,LightGBM_hapCat,2.713939,4.069071,8.249353,0.971496,0.928069,0.798767


## Neural network - DNN

In [47]:
import tensorflow as tf
from tensorflow import keras

2023-08-15 16:15:43.961180: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_12_1:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server:
2023-08-15 16:15:43.961264: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [48]:
tf.random.set_seed(20230516)

DL_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[X_train.shape[1]], name = 'input_layer'),
    keras.layers.Dense(100, activation="relu", name = 'hidden_layer1'),
    keras.layers.Dense(300, activation="relu", name = 'hidden_layer2'),
    keras.layers.Dense(1, name = 'output_layer')
])

DL_model.compile(loss="mean_squared_error",optimizer=tf.keras.optimizers.Adam(0.001)) 

# define the checkpoint
filepath = "/mnt/code/pred_2020/models/hap_dnn_2020.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = keras.callbacks.EarlyStopping(patience=8,restore_best_weights=True)
callbacks_list = [checkpoint,early_stopping]

# fit the model
history = DL_model.fit(X_train,y_train,epochs=50,validation_data = (X_val, y_val),callbacks=callbacks_list)


2023-08-15 16:16:37.386292: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-08-15 16:16:37.386960: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_12_1:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server:
2023-08-15 16:16:37.387006: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-15 16:16:37.387050: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (run-64db90884f722c0cf93dc7b0-2pqtb): /proc/driver/nvidia/version does not exist
2023-08-15 16:16:37.387365: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CP

Epoch 1/50
682/682 [==============================] - 23s 33ms/step - loss: 60.5095 - val_loss: 32.1668

Epoch 00001: val_loss improved from inf to 32.16682, saving model to /mnt/code/pred_2020/models/dnn_train_2020.h5
Epoch 2/50
682/682 [==============================] - 19s 28ms/step - loss: 32.2056 - val_loss: 31.8756

Epoch 00002: val_loss improved from 32.16682 to 31.87557, saving model to /mnt/code/pred_2020/models/dnn_train_2020.h5
Epoch 3/50
682/682 [==============================] - 19s 28ms/step - loss: 30.4800 - val_loss: 34.6853

Epoch 00003: val_loss did not improve from 31.87557
Epoch 4/50
682/682 [==============================] - 20s 29ms/step - loss: 26.4703 - val_loss: 30.9900

Epoch 00004: val_loss improved from 31.87557 to 30.99000, saving model to /mnt/code/pred_2020/models/dnn_train_2020.h5
Epoch 5/50
682/682 [==============================] - 20s 29ms/step - loss: 22.8002 - val_loss: 25.9009

Epoch 00005: val_loss improved from 30.99000 to 25.90093, saving model 

In [32]:
# load the saved neural network model
# DL_model = tf.keras.models.load_model('/mnt/code/pred_2020/models/hap_dnn_2020.h5')

In [53]:
# output prediction result
metrics = pred_rmse_corr(DL_model,'DNN_hapCat',X_train,y_train,
                         X_val,y_val,testHap,testPheno['YLD_BE_BLUP'],metrics)
metrics

2023-08-15 16:33:36.062617: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 19124412608 exceeds 10% of free system memory.
2023-08-15 16:34:06.207505: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 8100607376 exceeds 10% of free system memory.


,Method,RMSE_train,RMSE_val,RMSE_test,corr_train,corr_val,corr_test
0,LightGBM_hapCat,2.713939,4.069071,8.249353,0.971496,0.928069,0.798767
0,DNN_hapCat,3.487118,4.351017,8.083833,0.954041,0.918706,0.774208


# ElaticNet Model

In [54]:
from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import pickle

In [55]:
elsnt = ElasticNet(alpha=0.001, l1_ratio=0.25,random_state=seed)

# fit the model on training data
elsnt.fit(X_train, y_train)

# save the model to disk
filename = '/mnt/code/pred_2020/models/hap_elasticnet_2020.sav'
pickle.dump(elsnt, open(filename, 'wb'))

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 90672.18889941768, tolerance: 277.7539127609968
  model = cd_fast.enet_coordinate_descent(


In [56]:
# load the model
# filename = '/mnt/code/pred_2020/models/hap_elasticnet_2020.sav'
# elsnt = pickle.load(open(filename, 'rb'))

# run the model
metrics = pred_rmse_corr(elsnt,'elasticNet_obs',X_train,y_train,X_val,y_val,
                         testHap,testPheno['YLD_BE_BLUP'],metrics)
metrics

,Method,RMSE_train,RMSE_val,RMSE_test,corr_train,corr_val,corr_test
0,LightGBM_hapCat,2.713939,4.069071,8.249353,0.971496,0.928069,0.798767
0,DNN_hapCat,3.487118,4.351017,8.083833,0.954041,0.918706,0.774208
0,elasticNet_obs,3.193885,3.857174,8.999560,0.959152,0.935746,0.713221


In [77]:
testPheno['pred_hap_elsnt'] = elsnt.predict(testHap)
testPheno['pred_hap_lgbm'] = gbm.predict(testHap)
testPheno['pred_hap_dnn'] = DL_model.predict(testHap)

# Infinium marker

In [58]:
import pyreadr

# read in the marker data - train
result = pyreadr.read_r('/mnt/fpImp/train_fpGenoImp_hybrid.rds') # also works for RData
# done! 
# result is a dictionary where keys are the name of objects and the values python
# objects. In the case of Rds there is only one object with None as key
mkData = result[None] # extract the pandas data frame 

In [59]:
mkData.head()
mkData.shape

(33475, 41169)

In [60]:
# construct marker data for test and train data
X_test = mkData.loc[testPheno['LINE_NAME'],]
mkData = mkData.loc[trainPheno['LINE_NAME'],]

In [61]:
# splite the train data into train, validationa and test
seed = 20230510
np.random.seed(seed)
X_train, X_val, y_train, y_val = train_test_split(mkData, trainPheno['YLD_BE_BLUP'], test_size=0.1, random_state=seed)

In [62]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(21812, 41169)
(2424, 41169)
(9239, 41169)


# ElasticNet model

In [63]:
elsnt = ElasticNet(alpha=0.001, l1_ratio=0.25,random_state=seed)

# fit the model on training data
elsnt.fit(X_train, y_train)

# save the model to disk
filename = '/mnt/code/pred_2020/models/inf_elasticnet_2020.sav'
pickle.dump(elsnt, open(filename, 'wb'))

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 103048.43066260163, tolerance: 277.7539127609968
  model = cd_fast.enet_coordinate_descent(


In [65]:
# load the model
# filename = '/mnt/code/pred_2020/models/inf_elasticnet_2020.sav'
# elsnt = pickle.load(open(filename, 'rb'))

# run the model
metrics = pred_rmse_corr(elsnt,'elasticNet_inf',X_train,y_train,X_val,y_val,
                         X_test,testPheno['YLD_BE_BLUP'],metrics)
metrics

,Method,RMSE_train,RMSE_val,RMSE_test,corr_train,corr_val,corr_test
0,LightGBM_hapCat,2.713939,4.069071,8.249353,0.971496,0.928069,0.798767
0,DNN_hapCat,3.487118,4.351017,8.083833,0.954041,0.918706,0.774208
0,elasticNet_obs,3.193885,3.857174,8.999560,0.959152,0.935746,0.713221
0,elasticNet_inf,3.191491,3.825991,9.067572,0.959212,0.936813,0.711304


# LightGBM 

In [66]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(num_leaves=30,
                        learning_rate=0.1,
                        n_estimators=900,
                       bagging_fraction =  0.7,
                       feature_fraction = 0.5,
                       objective = "regression")

In [67]:
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='mse',
        callbacks=[lgb.early_stopping(5)])

# save model
gbm.booster_.save_model('/mnt/code/pred_2020/models/inf_lgbr_obs_30_900.txt')

# load model later
    #model = lightgbm.Booster(model_file='file.txt')
    #model.predict(predict[num_columns])
    

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 5.097132 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 186210
[LightGBM] [Info] Number of data points in the train set: 21812, number of used features: 40875
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_frac

In [68]:
# output prediction result
metrics = pred_rmse_corr(gbm,'LightGBM_inf',X_train,y_train,X_val,y_val,
                         X_test,testPheno['YLD_BE_BLUP'],metrics)
metrics

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7


,Method,RMSE_train,RMSE_val,RMSE_test,corr_train,corr_val,corr_test
0,LightGBM_hapCat,2.713939,4.069071,8.249353,0.971496,0.928069,0.798767
0,DNN_hapCat,3.487118,4.351017,8.083833,0.954041,0.918706,0.774208
0,elasticNet_obs,3.193885,3.857174,8.999560,0.959152,0.935746,0.713221
0,elasticNet_inf,3.191491,3.825991,9.067572,0.959212,0.936813,0.711304
0,LightGBM_inf,2.583351,4.276026,8.362801,0.974408,0.920226,0.793576


# DNN

In [ ]:
tf.random.set_seed(20230516)

DL_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[X_train.shape[1]], name = 'input_layer'),
    keras.layers.Dense(100, activation="relu", name = 'hidden_layer1'),
    keras.layers.Dense(300, activation="relu", name = 'hidden_layer2'),
    keras.layers.Dense(1, name = 'output_layer')
])

DL_model.compile(loss="mean_squared_error",optimizer=tf.keras.optimizers.Adam(0.001)) 

# define the checkpoint
filepath = "/mnt/code/pred_2020/models/inf_dnn_obs.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = keras.callbacks.EarlyStopping(patience=8,restore_best_weights=True)
callbacks_list = [checkpoint,early_stopping]

# fit the model
history = DL_model.fit(X_train,y_train,epochs=50,validation_data = (X_val, y_val),callbacks=callbacks_list)

Epoch 1/50
682/682 [==============================] - 10s 14ms/step - loss: 59.1135 - val_loss: 37.0281

Epoch 00001: val_loss improved from inf to 37.02813, saving model to /mnt/code/pred_2020/models/inf_dnn_obs.h5
Epoch 2/50
682/682 [==============================] - 8s 12ms/step - loss: 35.4609 - val_loss: 35.0848

Epoch 00002: val_loss improved from 37.02813 to 35.08479, saving model to /mnt/code/pred_2020/models/inf_dnn_obs.h5
Epoch 3/50
682/682 [==============================] - 8s 12ms/step - loss: 34.6222 - val_loss: 34.7943

Epoch 00003: val_loss improved from 35.08479 to 34.79431, saving model to /mnt/code/pred_2020/models/inf_dnn_obs.h5
Epoch 4/50
682/682 [==============================] - 8s 12ms/step - loss: 30.3198 - val_loss: 29.7951

Epoch 00004: val_loss improved from 34.79431 to 29.79510, saving model to /mnt/code/pred_2020/models/inf_dnn_obs.h5
Epoch 5/50
682/682 [==============================] - 8s 12ms/step - loss: 27.2064 - val_loss: 28.9439

Epoch 00005: val_los

In [70]:
# load the saved neural network model
# DL_model = tf.keras.models.load_model('/mnt/ML_HBLUP/NA_RM105_110_115/models/inf_dnn_obs.h5')
# output prediction result
metrics = pred_rmse_corr(DL_model,'DNN_inf',X_train,y_train,X_val,y_val,
                         X_test,testPheno['YLD_BE_BLUP'],metrics)
metrics

,Method,RMSE_train,RMSE_val,RMSE_test,corr_train,corr_val,corr_test
0,LightGBM_hapCat,2.713939,4.069071,8.249353,0.971496,0.928069,0.798767
0,DNN_hapCat,3.487118,4.351017,8.083833,0.954041,0.918706,0.774208
0,elasticNet_obs,3.193885,3.857174,8.999560,0.959152,0.935746,0.713221
0,elasticNet_inf,3.191491,3.825991,9.067572,0.959212,0.936813,0.711304
0,LightGBM_inf,2.583351,4.276026,8.362801,0.974408,0.920226,0.793576
0,DNN_inf,3.454239,4.409698,8.136475,0.953973,0.915222,0.779435


In [73]:
testPheno['pred_inf_elsnt'] = elsnt.predict(X_test)
testPheno['pred_inf_lgbm'] = gbm.predict(X_test)
testPheno['pred_inf_dnn'] = DL_model.predict(X_test)

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7


In [78]:
testPheno.head()

,LINE_NAME,FEMALE,MALE,YLD_BE_BLUP,MST_BE_BLUP,pred_hap_elsnt,pred_hap_lgbm,pred_inf_elsnt,pred_inf_lgbm,pred_inf_dnn,pred_hap_dnn
0,01DHD10+LH287,01DHD10,LH287,-15.661,-0.219,-11.380652,-8.602051,-10.990887,-8.858128,-3.705641,-6.430006
1,01DHD16+83INI14,01DHD16,83INI14,-41.839,-4.254,-35.774050,-26.850505,-46.533695,-27.879312,-32.630455,-25.352154
8,01DKD2+19HGZ1,01DKD2,19HGZ1,-5.917,0.091,-10.031257,-7.806338,-11.747356,-5.486347,-11.354566,-6.807303
9,01DKD2+80IDM2,01DKD2,80IDM2,-4.271,0.272,-9.682158,-3.608324,-9.204932,-2.773324,-6.941370,-6.797855
10,01DKD2+93IDI3,01DKD2,93IDI3,-14.926,0.210,-17.092687,-9.716184,-18.170208,-9.706514,-13.007492,-10.375535


In [79]:
testPheno.corr()

,YLD_BE_BLUP,MST_BE_BLUP,pred_hap_elsnt,pred_hap_lgbm,pred_inf_elsnt,pred_inf_lgbm,pred_inf_dnn,pred_hap_dnn
YLD_BE_BLUP,1.000000,0.408067,0.713221,0.798767,0.711304,0.793576,0.779435,0.774208
MST_BE_BLUP,0.408067,1.000000,0.448829,0.499593,0.453499,0.521477,0.473680,0.491369
pred_hap_elsnt,0.713221,0.448829,1.000000,0.846440,0.871783,0.839236,0.865681,0.886647
pred_hap_lgbm,0.798767,0.499593,0.846440,1.000000,0.843706,0.959050,0.914707,0.926854
pred_inf_elsnt,0.711304,0.453499,0.871783,0.843706,1.000000,0.845586,0.884750,0.871169
pred_inf_lgbm,0.793576,0.521477,0.839236,0.959050,0.845586,1.000000,0.916444,0.913621
pred_inf_dnn,0.779435,0.473680,0.865681,0.914707,0.884750,0.916444,1.000000,0.940339
pred_hap_dnn,0.774208,0.491369,0.886647,0.926854,0.871169,0.913621,0.940339,1.000000


In [80]:
testPheno.to_csv("pred_2020.csv")